# Running on new images
This notebook will walk you step by step through the process of using a pre-trained model to detect traffic signs in an image.

# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import os
import tensorflow as tf


from matplotlib import pyplot as plt
import glob as glob
import random
import pickle

import sys
import cv2
import csv
import re
%matplotlib inline

# tf.executing_eagerly()

In [ ]:
score_threshold = 0.3
np.random.seed(1)

# Environment setup

In [ ]:


sys.path.append('../models/research')  # Replace with the path to TensorFlow Object Detection API
sys.path.append('../darkflow')  # Replace with the path to Darkflow

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


## Model preparation 

# Tensorflow Object Detection API

In [ ]:
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous'
# MODEL_NAME = 'faster_rcnn_resnet_101'
# MODEL_NAME = 'faster_rcnn_resnet50'
# MODEL_NAME = 'faster_rcnn_inception_v2'
# MODEL_NAME = 'rfcn_resnet101'
# MODEL_NAME = 'ssd_inception_v2'
MODEL_NAME = 'ssd_mobilenet_v1'

In [ ]:
# Path to frozen detection graph. This is the actual model that is used for the traffic sign detection.
MODEL_PATH = os.path.join('models', MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'inference_graph/frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('gtsdb_data', 'gtsdb3_label_map.pbtxt')

NUM_CLASSES = 3


## Load a (frozen) Tensorflow model into memory

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `2`, we know that this corresponds to `mandatory`.

In [ ]:
PATH_TO_LABELS = r'D:\my_project\traffic-sign-detection\gtsdb_data\gtsdb3_label_map.pbtxt'

# Directly reading the file for troubleshooting
try:
    with open(PATH_TO_LABELS, 'rb') as f:
        print("Direct read:", f.read())
except Exception as e:
    print("File read error:", str(e))

# Your existing code
print('1')
try:
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
except Exception as e:
    print("Error during label map load:", str(e))
print('2')
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
print('3')
category_index = label_map_util.create_category_index(categories)
print(label_map)



## Helper code

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    #return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
    return np.array(image.getdata()).reshape((im_height, im_width, 3))

## Detection

In [ ]:

PATH_TO_TEST_IMAGES_DIR =  r'D:\my_project\traffic-sign-detection\test_images'

TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))

# Size, in inches, of the output images.
IMAGE_SIZE = (20, 20)

In [ ]:


def extract_number(pattern, filename):
    match = re.search(pattern, filename)
    if match:
        return int(match.group(1))
    else:
        return None

def extract_iteration(filename):
    # This will extract a one or two-digit iteration number after 'itr' and before '.jpg'
    return extract_number(r"itr(\d+).jpg", filename)
    

def extract_image_index(filename):
    # This will extract a three-digit image index between 'alg1_' and '_itr'
    return extract_number(r"alg1_(\d+)_itr", filename)

def save_adversarial_image_with_boxes(image, image_index, iteration, save_dir, algorithm_number, suffix="box"):
    image_with_boxes_name = f"{suffix}alg{algorithm_number}_{image_index:03d}_iter{iteration}.jpg"
    image_with_boxes_save_path = os.path.join(save_dir, image_with_boxes_name)
    plt.imsave(image_with_boxes_save_path, image)
    return image_with_boxes_name


def append_results(results, image_name, iteration, class_name, score, box, sum_diff, max_diff):
    results.append({
        'image_name': image_name,
        'iteration': iteration,
        'class': class_name,
        'score': float(score),
        'box': box,
        'total_perturbation': sum_diff,
        'max_perturbation': max_diff
    })
    
def save_results_to_csv(results, image_index, save_dir, algorithm_number):
    csv_filename = f"faster_rcnn_alg{algorithm_number}_results.csv"
    csv_path = os.path.join(save_dir, csv_filename)
    
    # Check if file exists to decide whether to write headers or not
    file_exists = os.path.isfile(csv_path)
    
    with open(csv_path, 'a', newline='') as csvfile:
        fieldnames = ['image_index', 'image_name', 'iteration', 'class', 'score', 'box', 'total_perturbation', 'max_perturbation']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # If the file does not exist, write the header
        if not file_exists:
            writer.writeheader()
        
        for result in results:
            # No need to increment image_index, it should be taken directly from the filename
            result['image_index'] = image_index
            writer.writerow(result)


In [ ]:

algorithm_number = 0

SAVE_DIR = "ssd_mobilenet_0"
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)


with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in TEST_IMAGE_PATHS:
            image_index = extract_image_index(image_path)
            iteration_number = extract_iteration(image_path)

            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            image_np_expanded = np.expand_dims(image_np, axis=0)

            results = []

            # Run the detection
            (boxes, scores, classes, num) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})

            # Process the detections and append results
            for i in range(boxes.shape[1]):
                if scores[0, i] > score_threshold:
                    class_id = int(classes[0, i])
                    class_name = category_index[class_id]['name']
                    box = boxes[0, i].tolist()
                    score = scores[0, i]
                    # Here you would calculate sum_diff and max_diff based on your application's needs
                    sum_diff = 0 # Placeholder, replace with actual calculation
                    max_diff = 0 # Placeholder, replace with actual calculation
                    image_name = f"alg{algorithm_number}_{image_index:03d}_itr{iteration_number}.jpg"
                    # Append the result for this detection
                    append_results(results, image_name, iteration_number, class_name, score, box, sum_diff, max_diff)

            # Save the image with boxes
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=6)
            
            save_adversarial_image_with_boxes(image_np, image_index, iteration_number, SAVE_DIR, algorithm_number)
            
            # Save results to CSV
            save_results_to_csv(results, image_index, SAVE_DIR, algorithm_number)

